In [1]:
%%time
import pandas as pd
import numpy
from collections import Counter
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('PTT_category_news.csv', sep='|')

# ckiplab word segment (中研院斷詞)
# Initialize drivers
# It takes time to download ckiplab models

# default參數是model="bert-base"
# ws = CkipWordSegmenter() 
# pos = CkipPosTagger()
# ner = CkipNerChunker()

# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny", device=0)
pos = CkipPosTagger(model="albert-tiny", device=0)
ner = CkipNerChunker(model="albert-tiny", device=0)


## Word Segmentation
tokens = ws(df.content)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(article_word, article_pos)) for article_word, article_pos in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.content)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
# allowPOS 過濾條件: 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)


def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.content:
    summary.append("暫無")
    sentiment.append("暫無")

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

# Save data to disk
df.to_csv('PTT_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("Tokenize OK!")

Inference: 100%|██████████| 3/3 [00:27<00:00,  9.30s/it]


Tokenize OK!
CPU times: total: 5min 39s
Wall time: 7min 40s


In [2]:
## Read it out 讀出看看
df = pd.read_csv('PTT_news_preprocessed.csv', sep='|')
df.head(1)


,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[('台積電', 5), ('原文', 3), ('技術', 3), ('日媒', 2), ...","['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定',...","['原文', '標題', '日媒', '台積電', '面板級', '規格', '尺寸', '...","[NerToken(word='日媒', ner='NORP', idx=(5, 7)), ...","[('原文', 'Na'), ('標題', 'Na'), ('：', 'COLONCATEG...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN


In [3]:
import pandas as pd
import numpy

In [4]:
df = pd.read_csv('PTT_category_news.csv', sep='|')

In [5]:
df.shape

(268, 7)

In [6]:
df.head()

,item_id,date,category,title,content,link,photo_link
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN
1,Stock_04-16_2,04-16,學術股市,Re: [新聞] 川普傳打算以關稅談判孤立中國 可能要各,從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元...,https://www.ptt.cc/bbs/Stock/M.1744794681.A.85...,NaN
2,Stock_04-16_3,04-16,學術股市,[新聞] 川普大讚「關稅政策」有效！ 發文激讚,川普大讚「關稅政策」有效！ 發文激讚：美國通膨下降了ETTODAY2025年04月16日 1...,https://www.ptt.cc/bbs/Stock/M.1744794813.A.00...,NaN
3,Stock_04-16_4,04-16,學術股市,[情報] 2834 臺企銀 113年股利:現金0.2+股票0.6,1. 標題：董事會通過擬議分派股利2. 來源：公開資訊觀測站3. 網址：4. 內文：1. 董...,https://www.ptt.cc/bbs/Stock/M.1744795089.A.D4...,NaN
4,Stock_04-16_5,04-16,學術股市,Re: [請益] 中國繼續擺爛是不是要害慘全世界？,感覺上你在反串不過中國的確在擺爛，而且早在川普發動關稅戰前就在擺爛中國要經濟轉型、要促進內需...,https://www.ptt.cc/bbs/Stock/M.1744795403.A.4D...,NaN


In [7]:
df.content[0:5]

0    原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...
1    從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元...
2    川普大讚「關稅政策」有效！　發文激讚：美國通膨下降了ETTODAY2025年04月16日 1...
3    1. 標題：董事會通過擬議分派股利2. 來源：公開資訊觀測站3. 網址：4. 內文：1. 董...
4    感覺上你在反串不過中國的確在擺爛，而且早在川普發動關稅戰前就在擺爛中國要經濟轉型、要促進內需...
Name: content, dtype: object

In [8]:
df['content'][0:5]

0    原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...
1    從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元...
2    川普大讚「關稅政策」有效！　發文激讚：美國通膨下降了ETTODAY2025年04月16日 1...
3    1. 標題：董事會通過擬議分派股利2. 來源：公開資訊觀測站3. 網址：4. 內文：1. 董...
4    感覺上你在反串不過中國的確在擺爛，而且早在川普發動關稅戰前就在擺爛中國要經濟轉型、要促進內需...
Name: content, dtype: object

In [9]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [10]:
# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")

In [11]:
%%time
# It takes time.
tokens = ws(df.content)

Inference: 100%|██████████| 3/3 [00:18<00:00,  6.21s/it]

CPU times: total: 1min 57s
Wall time: 19.7 s


In [12]:
len(tokens)

268

In [13]:
len(tokens[0])

198

In [14]:
tokens[0]

['原文',
 '標題',
 '：',
 '日媒',
 '：',
 '台積電',
 '將',
 '敲定',
 '面板級',
 '封裝',
 '規格',
 ' ',
 '先',
 '從',
 '較',
 '小',
 '尺寸',
 '開始',
 '原文',
 '連結',
 '：',
 '發布',
 '時間',
 '：',
 '2025-04-15 1',
 '4',
 ':19',
 '記者',
 '署名',
 '：',
 '劉忠勇',
 '原文',
 '內容',
 '：',
 '日媒',
 '報導',
 '，',
 '台積電',
 '（',
 '2330',
 '）',
 '即將',
 '敲定',
 '「',
 '面板級',
 '」',
 '先進',
 '封裝',
 '技術',
 '規格',
 '，',
 '預定',
 '最',
 '快',
 '2027年',
 '開始',
 '小量',
 '試產',
 '。據',
 '報導',
 '，',
 '台積電',
 '新',
 '封裝',
 '技術',
 '的',
 '第一',
 '代',
 '版本',
 '，',
 '將',
 '採用',
 '300x',
 '300 mm',
 '的',
 '方形',
 '基板',
 '，',
 '比',
 '先前',
 '試做',
 '的',
 '510',
 '×',
 '515 mm',
 '小',
 '得多',
 '，',
 '但',
 '相較於',
 '傳統',
 '的',
 '圓形晶圓',
 '可',
 '用',
 '面積',
 '更',
 '大',
 '。',
 '台積電',
 '為了',
 '嚴格',
 '控管',
 '品質',
 '，',
 '決定',
 '先',
 '採用',
 '略',
 '小',
 '的',
 '基板',
 '。',
 '為',
 '加快',
 '開發',
 '進度',
 '，',
 '台積電',
 '正在',
 '桃園',
 '興建',
 '試',
 '產線',
 '，',
 '目標',
 '是',
 '在',
 '2027年',
 '左',
 '右',
 '開始',
 '小量',
 '試產',
 '。',
 '。',
 '心得',
 '/',
 '評論',
 '：',
 '--',
 '南茂哥',
 '四年',
 '投資友',
 '達 ',
 

In [15]:
tokens[0][0]

'原文'

In [16]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 29/29 [03:12<00:00,  6.63s/it]


CPU times: total: 20min 26s
Wall time: 3min 14s


In [17]:
len(tokens_pos)

268

In [18]:
len(tokens_pos[0])

198

In [19]:
tokens_pos[0]

['Na',
 'Na',
 'COLONCATEGORY',
 'Na',
 'COLONCATEGORY',
 'Nb',
 'D',
 'VE',
 'Na',
 'VC',
 'Na',
 'WHITESPACE',
 'D',
 'P',
 'Dfa',
 'VH',
 'Na',
 'VL',
 'Na',
 'VC',
 'COLONCATEGORY',
 'VC',
 'Na',
 'COLONCATEGORY',
 'Neu',
 'Neu',
 'COLONCATEGORY',
 'Na',
 'VC',
 'COLONCATEGORY',
 'Nb',
 'Na',
 'Na',
 'COLONCATEGORY',
 'Na',
 'VE',
 'COMMACATEGORY',
 'Nb',
 'PARENTHESISCATEGORY',
 'Neu',
 'PARENTHESISCATEGORY',
 'D',
 'VE',
 'PARENTHESISCATEGORY',
 'Na',
 'PARENTHESISCATEGORY',
 'VH',
 'VC',
 'Na',
 'Na',
 'COMMACATEGORY',
 'VE',
 'Dfa',
 'VH',
 'Neu',
 'VL',
 'VH',
 'VA',
 'PERIODCATEGORY',
 'VE',
 'COMMACATEGORY',
 'Nb',
 'VH',
 'VC',
 'Na',
 'DE',
 'Neu',
 'Na',
 'Na',
 'COMMACATEGORY',
 'D',
 'VC',
 'Neu',
 'Neu',
 'DE',
 'Na',
 'Na',
 'COMMACATEGORY',
 'P',
 'Nd',
 'VF',
 'DE',
 'FW',
 'FW',
 'Neu',
 'VH',
 'Dfb',
 'COMMACATEGORY',
 'Cbb',
 'VB',
 'VH',
 'DE',
 'Na',
 'A',
 'P',
 'Na',
 'Dfa',
 'VH',
 'PERIODCATEGORY',
 'Nb',
 'P',
 'VH',
 'VC',
 'Na',
 'COMMACATEGORY',
 'VE',


In [20]:
for article_word, article_pos in zip(tokens,tokens_pos):
    print(article_word)
    print(article_pos)

['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定', '面板級', '封裝', '規格', ' ', '先', '從', '較', '小', '尺寸', '開始', '原文', '連結', '：', '發布', '時間', '：', '2025-04-15 1', '4', ':19', '記者', '署名', '：', '劉忠勇', '原文', '內容', '：', '日媒', '報導', '，', '台積電', '（', '2330', '）', '即將', '敲定', '「', '面板級', '」', '先進', '封裝', '技術', '規格', '，', '預定', '最', '快', '2027年', '開始', '小量', '試產', '。據', '報導', '，', '台積電', '新', '封裝', '技術', '的', '第一', '代', '版本', '，', '將', '採用', '300x', '300 mm', '的', '方形', '基板', '，', '比', '先前', '試做', '的', '510', '×', '515 mm', '小', '得多', '，', '但', '相較於', '傳統', '的', '圓形晶圓', '可', '用', '面積', '更', '大', '。', '台積電', '為了', '嚴格', '控管', '品質', '，', '決定', '先', '採用', '略', '小', '的', '基板', '。', '為', '加快', '開發', '進度', '，', '台積電', '正在', '桃園', '興建', '試', '產線', '，', '目標', '是', '在', '2027年', '左', '右', '開始', '小量', '試產', '。', '。', '心得', '/', '評論', '：', '--', '南茂哥', '四年', '投資友', '達 ', '資產', '快腰斬 ', '大', '獲', '全', '勝', '啦', '！', '有', '夠', '慘', '的', '不少', '小白', '推文', '說', '很', '棒', '喔', '！', '真', '是', '害死', '不少', '人', '第一', ' 群',

In [21]:
word_pos_pair = [list(zip(article_word,article_pos)) for article_word, article_pos in zip(tokens, tokens_pos)]

In [22]:
len(word_pos_pair)

268

In [23]:
word_pos_pair

[[('原文', 'Na'),
  ('標題', 'Na'),
  ('：', 'COLONCATEGORY'),
  ('日媒', 'Na'),
  ('：', 'COLONCATEGORY'),
  ('台積電', 'Nb'),
  ('將', 'D'),
  ('敲定', 'VE'),
  ('面板級', 'Na'),
  ('封裝', 'VC'),
  ('規格', 'Na'),
  (' ', 'WHITESPACE'),
  ('先', 'D'),
  ('從', 'P'),
  ('較', 'Dfa'),
  ('小', 'VH'),
  ('尺寸', 'Na'),
  ('開始', 'VL'),
  ('原文', 'Na'),
  ('連結', 'VC'),
  ('：', 'COLONCATEGORY'),
  ('發布', 'VC'),
  ('時間', 'Na'),
  ('：', 'COLONCATEGORY'),
  ('2025-04-15 1', 'Neu'),
  ('4', 'Neu'),
  (':19', 'COLONCATEGORY'),
  ('記者', 'Na'),
  ('署名', 'VC'),
  ('：', 'COLONCATEGORY'),
  ('劉忠勇', 'Nb'),
  ('原文', 'Na'),
  ('內容', 'Na'),
  ('：', 'COLONCATEGORY'),
  ('日媒', 'Na'),
  ('報導', 'VE'),
  ('，', 'COMMACATEGORY'),
  ('台積電', 'Nb'),
  ('（', 'PARENTHESISCATEGORY'),
  ('2330', 'Neu'),
  ('）', 'PARENTHESISCATEGORY'),
  ('即將', 'D'),
  ('敲定', 'VE'),
  ('「', 'PARENTHESISCATEGORY'),
  ('面板級', 'Na'),
  ('」', 'PARENTHESISCATEGORY'),
  ('先進', 'VH'),
  ('封裝', 'VC'),
  ('技術', 'Na'),
  ('規格', 'Na'),
  ('，', 'COMMACATEGORY'),
  ('預定', '

In [24]:
a=[1,2,3]
b=['a','b','c']
ab=zip(a,b)
list(ab)

[(1, 'a'), (2, 'b'), (3, 'c')]

In [25]:
# list(zip(w,p)) for w, p in zip(tokens, tokens_pos)
a=[[1,2],[3,4],[5,6]]
b=[['a','b'],['c','d'],['e','f']]
ab=zip(a,b)
list(ab)


[([1, 2], ['a', 'b']), ([3, 4], ['c', 'd']), ([5, 6], ['e', 'f'])]

In [26]:
for item1, item2 in zip(a, b):
    print(item1,'with', item2)

[1, 2] with ['a', 'b']
[3, 4] with ['c', 'd']
[5, 6] with ['e', 'f']


In [27]:
for item1, item2 in zip(a, b):
    print(item1,'with', item2)
    print(list(zip(item1, item2)))
    print('---')

[1, 2] with ['a', 'b']
[(1, 'a'), (2, 'b')]
---
[3, 4] with ['c', 'd']
[(3, 'c'), (4, 'd')]
---
[5, 6] with ['e', 'f']
[(5, 'e'), (6, 'f')]
---


In [28]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 3/3 [00:13<00:00,  4.41s/it]


CPU times: total: 1min 28s
Wall time: 14.6 s


In [29]:
entity_list[0]

[NerToken(word='日媒', ner='NORP', idx=(5, 7)),
 NerToken(word='劉忠勇', ner='PERSON', idx=(61, 64)),
 NerToken(word='積電', ner='ORG', idx=(75, 77)),
 NerToken(word='2027年', ner='DATE', idx=(105, 110)),
 NerToken(word='台積', ner='ORG', idx=(121, 123)),
 NerToken(word='第一', ner='ORDINAL', idx=(130, 132)),
 NerToken(word='300x300', ner='CARDINAL', idx=(139, 146)),
 NerToken(word='台積電', ner='ORG', idx=(193, 196)),
 NerToken(word='桃園', ner='GPE', idx=(229, 231)),
 NerToken(word='2027年', ner='DATE', idx=(241, 246)),
 NerToken(word='四年', ner='DATE', idx=(267, 269)),
 NerToken(word='第一', ner='ORDINAL', idx=(308, 310)),
 NerToken(word='第二', ner='ORDINAL', idx=(316, 318))]

In [30]:
# 過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

In [31]:
tokens_v2[0]

['原文',
 '標題',
 '日媒',
 '台積電',
 '面板級',
 '規格',
 '尺寸',
 '原文',
 '時間',
 '記者',
 '劉忠勇',
 '原文',
 '內容',
 '日媒',
 '台積電',
 '面板級',
 '技術',
 '規格',
 '台積電',
 '技術',
 '版本',
 '方形',
 '基板',
 '圓形晶圓',
 '面積',
 '台積電',
 '品質',
 '基板',
 '進度',
 '台積電',
 '桃園',
 '產線',
 '目標',
 '心得',
 '評論',
 '南茂哥',
 '達 ',
 '資產',
 '小白',
 '推文',
 '技術',
 '貢獻',
 '營收',
 '產能',
 '轉型',
 '半導體']

In [32]:
df['tokens'] = tokens

In [33]:
df['tokens_v2'] = tokens_v2

In [34]:
df['entities'] = entity_list

In [35]:
df['token_pos'] = word_pos_pair

In [36]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN,"[原文, 標題, ：, 日媒, ：, 台積電, 將, 敲定, 面板級, 封裝, 規格, ,...","[原文, 標題, 日媒, 台積電, 面板級, 規格, 尺寸, 原文, 時間, 記者, 劉忠勇...","[(日媒, NORP, (5, 7)), (劉忠勇, PERSON, (61, 64)), ...","[(原文, Na), (標題, Na), (：, COLONCATEGORY), (日媒, ..."


In [37]:
# allowPOS過濾條件:兩個字以上與特定的詞性
# allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']
allowPOS=['Na','Nb','Nc']

from collections import Counter
def word_frequency( wp_pair ):
    filtered_words =[]
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter( filtered_words)
    return counter.most_common(20)

In [38]:
word_pos_pair[0]

[('原文', 'Na'),
 ('標題', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日媒', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('台積電', 'Nb'),
 ('將', 'D'),
 ('敲定', 'VE'),
 ('面板級', 'Na'),
 ('封裝', 'VC'),
 ('規格', 'Na'),
 (' ', 'WHITESPACE'),
 ('先', 'D'),
 ('從', 'P'),
 ('較', 'Dfa'),
 ('小', 'VH'),
 ('尺寸', 'Na'),
 ('開始', 'VL'),
 ('原文', 'Na'),
 ('連結', 'VC'),
 ('：', 'COLONCATEGORY'),
 ('發布', 'VC'),
 ('時間', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('2025-04-15 1', 'Neu'),
 ('4', 'Neu'),
 (':19', 'COLONCATEGORY'),
 ('記者', 'Na'),
 ('署名', 'VC'),
 ('：', 'COLONCATEGORY'),
 ('劉忠勇', 'Nb'),
 ('原文', 'Na'),
 ('內容', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日媒', 'Na'),
 ('報導', 'VE'),
 ('，', 'COMMACATEGORY'),
 ('台積電', 'Nb'),
 ('（', 'PARENTHESISCATEGORY'),
 ('2330', 'Neu'),
 ('）', 'PARENTHESISCATEGORY'),
 ('即將', 'D'),
 ('敲定', 'VE'),
 ('「', 'PARENTHESISCATEGORY'),
 ('面板級', 'Na'),
 ('」', 'PARENTHESISCATEGORY'),
 ('先進', 'VH'),
 ('封裝', 'VC'),
 ('技術', 'Na'),
 ('規格', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('預定', 'VE'),
 ('最', 'Dfa'),
 ('快', 'VH'),
 ('2027年', 'Neu')

In [39]:
word_frequency(word_pos_pair[0])

[('台積電', 5),
 ('原文', 3),
 ('技術', 3),
 ('日媒', 2),
 ('面板級', 2),
 ('規格', 2),
 ('基板', 2),
 ('標題', 1),
 ('尺寸', 1),
 ('時間', 1),
 ('記者', 1),
 ('劉忠勇', 1),
 ('內容', 1),
 ('版本', 1),
 ('方形', 1),
 ('圓形晶圓', 1),
 ('面積', 1),
 ('品質', 1),
 ('進度', 1),
 ('桃園', 1)]

In [40]:
word_frequency(word_pos_pair[1])

[('中國', 36),
 ('美國', 30),
 ('國家', 21),
 ('技術', 13),
 ('位置', 12),
 ('科技', 9),
 ('關稅', 8),
 ('產業', 8),
 ('晶片', 8),
 ('總統', 8),
 ('台灣', 8),
 ('程度', 8),
 ('世界', 7),
 ('政策', 7),
 ('經濟', 7),
 ('產品', 6),
 ('廣場', 5),
 ('政府', 5),
 ('傷害', 5),
 ('局勢', 5)]

In [41]:
word_frequency(word_pos_pair[2])

[('關稅', 9),
 ('川普', 6),
 ('美國', 4),
 ('政策', 3),
 ('記者', 2),
 ('總統', 2),
 ('發文', 2),
 ('全球', 2),
 ('產業', 2),
 ('通膨', 1),
 ('吳美依', 1),
 ('貨膨脹', 1),
 ('市場', 1),
 ('自家', 1),
 ('社群', 1),
 ('媒體', 1),
 ('川普14日', 1),
 ('白宮', 1),
 ('薩爾瓦多', 1),
 ('布格磊', 1)]

In [42]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)  

CPU times: total: 15.6 ms
Wall time: 16.8 ms


In [43]:
keyfreqs[0:2]

[[('台積電', 5),
  ('原文', 3),
  ('技術', 3),
  ('日媒', 2),
  ('面板級', 2),
  ('規格', 2),
  ('基板', 2),
  ('標題', 1),
  ('尺寸', 1),
  ('時間', 1),
  ('記者', 1),
  ('劉忠勇', 1),
  ('內容', 1),
  ('版本', 1),
  ('方形', 1),
  ('圓形晶圓', 1),
  ('面積', 1),
  ('品質', 1),
  ('進度', 1),
  ('桃園', 1)],
 [('中國', 36),
  ('美國', 30),
  ('國家', 21),
  ('技術', 13),
  ('位置', 12),
  ('科技', 9),
  ('關稅', 8),
  ('產業', 8),
  ('晶片', 8),
  ('總統', 8),
  ('台灣', 8),
  ('程度', 8),
  ('世界', 7),
  ('政策', 7),
  ('經濟', 7),
  ('產品', 6),
  ('廣場', 5),
  ('政府', 5),
  ('傷害', 5),
  ('局勢', 5)]]

In [44]:
df['top_key_freq'] = keyfreqs

In [45]:
df.head(2)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN,"[原文, 標題, ：, 日媒, ：, 台積電, 將, 敲定, 面板級, 封裝, 規格, ,...","[原文, 標題, 日媒, 台積電, 面板級, 規格, 尺寸, 原文, 時間, 記者, 劉忠勇...","[(日媒, NORP, (5, 7)), (劉忠勇, PERSON, (61, 64)), ...","[(原文, Na), (標題, Na), (：, COLONCATEGORY), (日媒, ...","[(台積電, 5), (原文, 3), (技術, 3), (日媒, 2), (面板級, 2)..."
1,Stock_04-16_2,04-16,學術股市,Re: [新聞] 川普傳打算以關稅談判孤立中國 可能要各,從廣場協議看2018美中貿易戰 的升級版，及這次的關稅事件，廣場協議的主因也是貿易逆差跟美元...,https://www.ptt.cc/bbs/Stock/M.1744794681.A.85...,NaN,"[從, 廣場, 協議, 看, 2018, 美中, 貿易戰 , 的, 升級版, ，, 及, 這...","[廣場, 協議, 美中, 貿易戰 , 升級版, 關稅, 事件, 廣場, 協議, 主因, 貿易...","[(美國鋼鐵, ORG, (57, 61)), (第一, ORDINAL, (78, 80)...","[(從, P), (廣場, Nc), (協議, Na), (看, VC), (2018, N...","[(中國, 36), (美國, 30), (國家, 21), (技術, 13), (位置, ..."


In [46]:
df.iloc[0].top_key_freq

[('台積電', 5),
 ('原文', 3),
 ('技術', 3),
 ('日媒', 2),
 ('面板級', 2),
 ('規格', 2),
 ('基板', 2),
 ('標題', 1),
 ('尺寸', 1),
 ('時間', 1),
 ('記者', 1),
 ('劉忠勇', 1),
 ('內容', 1),
 ('版本', 1),
 ('方形', 1),
 ('圓形晶圓', 1),
 ('面積', 1),
 ('品質', 1),
 ('進度', 1),
 ('桃園', 1)]

In [47]:
%%time
summary=[]
sentiment=[]
for text in df.content: # process piece by piece
    summary.append("暫無")  
    sentiment.append("暫無")

CPU times: total: 0 ns
Wall time: 0 ns


In [48]:
df['summary'] = summary

In [49]:
df['sentiment'] = sentiment

In [50]:
df.head(1)

,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq,summary,sentiment
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN,"[原文, 標題, ：, 日媒, ：, 台積電, 將, 敲定, 面板級, 封裝, 規格, ,...","[原文, 標題, 日媒, 台積電, 面板級, 規格, 尺寸, 原文, 時間, 記者, 劉忠勇...","[(日媒, NORP, (5, 7)), (劉忠勇, PERSON, (61, 64)), ...","[(原文, Na), (標題, Na), (：, COLONCATEGORY), (日媒, ...","[(台積電, 5), (原文, 3), (技術, 3), (日媒, 2), (面板級, 2)...",暫無,暫無


In [51]:
df.columns

Index(['item_id', 'date', 'category', 'title', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq',
       'summary', 'sentiment'],
      dtype='object')

In [52]:
# This operation is the same as slicing
df=df[['item_id', 'date', 'category', 'title', 'content','sentiment', 'summary', 'top_key_freq', 'tokens',
       'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link'
       ]]

In [53]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[(台積電, 5), (原文, 3), (技術, 3), (日媒, 2), (面板級, 2)...","[原文, 標題, ：, 日媒, ：, 台積電, 將, 敲定, 面板級, 封裝, 規格, ,...","[原文, 標題, 日媒, 台積電, 面板級, 規格, 尺寸, 原文, 時間, 記者, 劉忠勇...","[(日媒, NORP, (5, 7)), (劉忠勇, PERSON, (61, 64)), ...","[(原文, Na), (標題, Na), (：, COLONCATEGORY), (日媒, ...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN


In [54]:
df.to_csv('PTT_news_preprocessed.csv', sep='|', index=False)

In [55]:
df = pd.read_csv('PTT_news_preprocessed.csv', sep='|')

In [56]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,Stock_04-16_1,04-16,學術股市,[新聞] 日媒：台積電將敲定面板級封裝規格 先從,原文標題：日媒：台積電將敲定面板級封裝規格 先從較小尺寸開始原文連結：發布時間：2025-0...,暫無,暫無,"[('台積電', 5), ('原文', 3), ('技術', 3), ('日媒', 2), ...","['原文', '標題', '：', '日媒', '：', '台積電', '將', '敲定',...","['原文', '標題', '日媒', '台積電', '面板級', '規格', '尺寸', '...","[NerToken(word='日媒', ner='NORP', idx=(5, 7)), ...","[('原文', 'Na'), ('標題', 'Na'), ('：', 'COLONCATEG...",https://www.ptt.cc/bbs/Stock/M.1744794518.A.3D...,NaN
